In [1]:
import os
import sys

MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/Base',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [2]:
# import
import numpy
from config import *
from optim import Adam
from models import CnnModelReg

# for time series split
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3MB 1.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
# configuration setting
def model_config():
    # parameter for CNN Model
    filter_num = [30]
    filter_size = [3]
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in filter_num:
        for j in filter_size:
            for k in epochs:
                for l in batch_size:
                    for m in learning_rate:
                        config = [i, j, k, l, m]
                        configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_filter, n_fsize, n_epochs, n_batch, learning_rate = config
    cnn_params = {'filter_num':n_filter, 'filter_size':n_fsize, 
                  'stride':1, 'padding':0}
    model = CnnModelReg(input_dim=(1, 24, 9),params=cnn_params)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [4]:
import pandas as pd
import numpy
import time
from datetime import datetime

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00247/'
data_name = 'data_akbilgic.xlsx'
df = pd.read_excel(data_url+data_name, header=1)
df.drop(columns=df.columns[[0]], axis=1, inplace=True)
cols = df.columns.tolist()
cols = cols[3:]+cols[:3]
df = df[cols]
df.head()

,DAX,FTSE,NIKKEI,BOVESPA,EU,EM,ISE,ISE.1,SP
0,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376,-0.004679
1,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,0.007787
2,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469
3,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391
4,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533


In [11]:
from scipy.stats import skew, kurtosis
from statsmodels.tsa.stattools import adfuller

# jb = (n/6)*(skewness**2 + (kurtosis**2/4))

def data_statistics(df):
    df = df.dropna()
    data = df.values
    num = len(df)
    skewness_ = skew(data)
    kurtosis_ = kurtosis(data)
    jarque_bera_ = (num/6)*(skewness_**2 + (kurtosis_**2/4))
    result = adfuller(data)
    adf_ = result[0]
    print(f'skewness : {skewness_}')
    print(f'kurtosis : {kurtosis_}')
    print(f'jarque bera : {jarque_bera_}')
    print(f'ADF : {adf_}')

data_statistics(df['DAX'])

skewness : -0.10698158341639125
kurtosis : 2.1040646403491836
jarque bera : 99.89405752814437
ADF : -23.056515731416432


In [6]:
df.describe()

,DAX,FTSE,NIKKEI,BOVESPA,EU,EM,ISE,ISE.1,SP
count,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000,536.000000
mean,0.000721,0.000510,0.000308,0.000935,0.000471,0.000936,0.001629,0.001552,0.000643
std,0.014557,0.012656,0.014850,0.015751,0.012990,0.010501,0.016264,0.021122,0.014093
min,-0.052331,-0.054816,-0.050448,-0.053849,-0.048817,-0.038564,-0.062208,-0.084716,-0.054262
25%,-0.006212,-0.005808,-0.007407,-0.007215,-0.005952,-0.004911,-0.006669,-0.009753,-0.004675
50%,0.000887,0.000409,0.000000,0.000279,0.000196,0.001077,0.002189,0.002643,0.000876
75%,0.008224,0.007428,0.007882,0.008881,0.007792,0.006423,0.010584,0.013809,0.006706
max,0.058951,0.050323,0.061229,0.063792,0.067042,0.047805,0.068952,0.100621,0.068366


In [7]:
df.isnull().sum()

DAX        0
FTSE       0
NIKKEI     0
BOVESPA    0
EU         0
EM         0
ISE        0
ISE.1      0
SP         0
dtype: int64

In [8]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 9
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var1(t-23),var2(t-23),var3(t-23),var4(t-23),var5(t-23),var6(t-23),var7(t-23),var8(t-23),var9(t-23),var1(t-22),var2(t-22),var3(t-22),var4(t-22),var5(t-22),var6(t-22),var7(t-22),var8(t-22),var9(t-22),var1(t-21),var2(t-21),var3(t-21),var4(t-21),var5(t-21),var6(t-21),var7(t-21),var8(t-21),var9(t-21),var1(t-20),var2(t-20),var3(t-20),var4(t-20),...,var7(t-5),var8(t-5),var9(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var7(t-4),var8(t-4),var9(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var7(t-3),var8(t-3),var9(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var8(t-2),var9(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var1(t)
24,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376,-0.004679,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,0.007787,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391,-0.019873,-0.012710,-0.004474,-0.009764,...,-0.008777,-0.023458,-0.023053,-0.015637,-0.017454,-0.015134,-0.016289,-0.019739,-0.019091,-0.025919,-0.035607,-0.000533,0.024040,0.021039,-0.006175,0.027574,0.017862,0.012719,0.015279,0.022403,0.015710,0.026577,0.015275,0.026908,0.009565,0.018770,0.015166,0.018578,0.023231,-0.007518,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.016233,0.029306
25,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,0.007787,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533,-0.013526,-0.005026,-0.049039,-0.053849,...,-0.025919,-0.035607,-0.000533,0.024040,0.021039,-0.006175,0.027574,0.017862,0.012719,0.015279,0.022403,0.015710,0.026577,0.015275,0.026908,0.009565,0.018770,0.015166,0.018578,0.023231,-0.007518,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.016233,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.026541,0.004766
26,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,-0.030469,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533,-0.013526,-0.005026,-0.049039,-0.053849,-0.012451,-0.022630,-0.029191,-0.042361,-0.022823,-0.017674,-0.006141,0.000000,0.003572,...,0.015279,0.022403,0.015710,0.026577,0.015275,0.026908,0.009565,0.018770,0.015166,0.018578,0.023231,-0.007518,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.016233,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.026541,0.004766,0.003651,-0.013411,-0.015462,0.005627,0.007895,0.011353,0.021468,0.001484,-0.035170
27,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,0.003391,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.021533,-0.013526,-0.005026,-0.049039,-0.053849,-0.012451,-0.022630,-0.029191,-0.042361,-0.022823,-0.017674,-0.006141,0.000000,0.003572,-0.012220,-0.004827,0.015445,-0.000272,0.001757,-0.047383,-0.050945,0.002912,-0.040302,...,0.018578,0.023231,-0.007518,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.016233,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.026541,0.004766,0.003651,-0.013411,-0.015462,0.005627,0.007895,0.011353,0.021468,0.001484,-0.035170,-0.022182,-0.002902,-0.021440,-0.024388,-0.002139,-0.040542,-0.043907,-0.050369,0.005434
28,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.0215

In [9]:
n_splits = 3
test_size = (int)(len(new_df)*0.2)
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs, test_size=test_size).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []

i = 1

print('config : filter_num, filter_size, epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        predicted = model.predict(val_X)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f'error(rmse):{error.item():.2f}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    # outer
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    predicted = selected_model.predict(test_cv_X)
    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}')
    print()

config : filter_num, filter_size, epochs, batch_size, learning_rate
fold : 1/3
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):23.05
 == train [30, 3, 30, 64, 0.001] model ==  error(rmse):4.11
train-size:80, val-size:102, test-size:102
best_model => error(rmse) : 4.50, param:[30, 3, 30, 64, 0.001]

fold : 2/3
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):74.63
 == train [30, 3, 30, 64, 0.001] model ==  error(rmse):3.06
train-size:182, val-size:102, test-size:102
best_model => error(rmse) : 3.68, param:[30, 3, 30, 64, 0.001]

fold : 3/3
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):0.44
 == train [30, 3, 30, 64, 0.001] model ==  error(rmse):3.96
train-size:284, val-size:102, test-size:102
best_model => error(rmse) : 50.67, param:[30, 3, 30, 64, 0.01]



In [10]:
predicted = selected_model.predict(test_cv_X)

def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

MSE: mean=867.0400830503613, std=1202.3021681885648
RMSE: mean=19.615783454149422, std=21.960444497556946
MAE: mean=19.607475275414302, std=21.956540043747722
